# annotation_videos

* classify_videoで動画の2値分類をするため、教師ラベルを人力で付与します
    * 今回は歌動画を1, 非歌動画を0でラベリングしています
* アノテーションしたデータを元に改めて訓練用のテーブルデータを作成します

In [2]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# 画像の保存
import requests

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [7]:
# 取得した全動画情報を抽出する
file_select = 'okakoro'
# file_select = 'hololive'
file_names = glob.glob('output/videos/'+file_select+'/*')

# 日本語用の分類器を利用する場合、英語圏の動画は除外する
# eng_file_names = [
#     'Airani Iofifteen_videos.csv', 
#     'Anya Melfissa_videos.csv', 
#     'Ayunda Risu_videos.csv', 
#     'Gawr Gura_videos.csv', 
#     'hololive Indonesia_videos.csv', 
#     'Kureiji Ollie_videos.csv', 
#     'Moona Hoshinova_videos.csv', 
#     'Mori Calliope_videos.csv', 
#     'Ninomae Inanis_videos.csv', 
#     'Pavolia Reine_videos.csv', 
#     'Takanashi Kiara_videos.csv', 
#     'Watson Amelia_videos.csv'
# ]

# for eng_file_name in eng_file_names:
#     if eng_file_name in file_names:
#         file_names.remove('output/videos/'+file_select+'/'+eng_file_name)

for i, file_name in enumerate(file_names):
    videos = pd.read_csv(file_name, index_col=0)
    # テーブルを結合する
    if i == 0:
        all_videos = videos
    else:
        all_videos = pd.concat([all_videos, videos])

# DescriptionがNanのケースがあるため補完する
all_videos = all_videos.fillna('Description=Nan')
# indexを振り直す
all_videos = all_videos.reset_index()
# 変なラベルが付与されているのを整理
all_videos = all_videos.drop(['index'], axis=1)

print("全動画数:", all_videos.shape[0])

全動画数: 951


In [13]:
# サムネイルを表示する
def output_html(videos, top_n=10):
    if top_n <= 0:
        top_n = videos.shape[0]
        
    indexes = np.arange(videos.shape[0])[0:top_n]        
    html = '<h1>動画一覧を表示</h1>'
    html += '<div style="float:left;">'
    for i in indexes:
        html += ('<img src="'+np.array(videos['Thumbnail'])[i] +' "alt="取得できませんでした" width="100">')
        html += ('<a href="https://www.youtube.com/watch?v='+np.array(videos['Id'])[i]+'">#'+str(i+1)+'. '+np.array(videos['Title'])[i]+'</a><br>')
        html += (' Label='+str(np.array(videos['Label'])[i])+'<br>')
    html += '</div>'
    return html

In [9]:
# 再生時間によるフィルタリング、1分〜8分に設定（1コーラスを考慮）
filetered_videos = all_videos[np.logical_and(all_videos['Duration']<480, all_videos['Duration']>60)]
print("再生時間によるフィルタリング後：", filetered_videos.shape[0])
# HTML(output_html(filetered_videos, top_n=10))

再生時間によるフィルタリング後： 44


In [10]:
# サムネイルをunlabeledディレクトリに保存
os.makedirs('output/thumbnails/'+file_select+'/unlabeled/', exist_ok=True)
os.makedirs('output/thumbnails/'+file_select+'/label_0/', exist_ok=True)
os.makedirs('output/thumbnails/'+file_select+'/label_1/', exist_ok=True)

thumbnails = np.array(filetered_videos['Thumbnail'])
video_ids = np.array(filetered_videos['Id'])
video_ids_label_0, video_ids_label_1 = [], []
for file_name in glob.glob('output/thumbnails/'+file_select+'/label_0/*'):
    file_name = file_name.replace('output/thumbnails/'+file_select+'/label_0/', '').replace('.png', '')
    video_ids_label_0.append(file_name)
for file_name in glob.glob('output/thumbnails/'+file_select+'/label_1/*'):
    file_name = file_name.replace('output/thumbnails/'+file_select+'/label_1/', '').replace('.png', '')
    video_ids_label_1.append(file_name)
video_ids_label_0 = np.array(video_ids_label_0)
video_ids_label_1 = np.array(video_ids_label_1)
    
for i, thumbnail in enumerate(thumbnails):
    response = requests.get(thumbnail)
    image = response.content
    video_id = video_ids[i]
    
    # label_0, label_1に保存済みのデータであれば保存しない
    if video_id in video_ids_label_0 or video_id in video_ids_label_1:
        pass
    else:
        file_name = 'output/thumbnails/'+file_select+'/unlabeled/'+video_ids[i]+'.png'
        with open(file_name, "wb") as tmp:
            tmp.write(image)

* unlabeledに格納されたデータをlabel_0とlabel_1に分類（手動）

In [11]:
video_ids = np.array(all_videos['Id'])
concatenated_videos_tmp = []
for label in range(2):
    # アノテーションされたデータを再収集
    video_ids_labeled = glob.glob('output/thumbnails/'+file_select+'/label_'+str(label)+'/*')
    for i, video_id in enumerate(video_ids_labeled):
        video_id = video_id.replace('output/thumbnails/'+file_select+'/label_'+str(label)+'/', '')
        video_id = video_id.replace('.png', '')
        # 元のデータセットにある場合にデータセットを取り出す
        if video_id in video_ids:
            video = all_videos.loc[all_videos['Id']==video_id]
            video.loc[:, 'Label'] = label
            concatenated_videos_tmp.append(video)

# アノテーションされていないデータを再収集
video_ids_unlabeled = glob.glob('output/thumbnails/'+file_select+'/unlabeled/*')
for i, video_id in enumerate(video_ids_unlabeled):
    video_id = video_id.replace('output/thumbnails/'+file_select+'/unlabeled/', '')
    video_id = video_id.replace('.png', '')
    if video_id in video_ids:
        video = all_videos.loc[all_videos['Id']==video_id]
        video.loc[:, 'Label'] = -1
        concatenated_videos_tmp.append(video)
            
for i, video in enumerate(concatenated_videos_tmp):
    if i == 0:
        concatenated_videos = video
    else:
        concatenated_videos = pd.concat([concatenated_videos, video])

concatenated_videos = concatenated_videos.reset_index()
concatenated_videos = concatenated_videos.drop(['index'], axis=1)
concatenated_videos.to_csv('output/train/'+file_select+'/videos.csv')

print("再生時間によるフィルタリング後：", concatenated_videos.shape[0])

再生時間によるフィルタリング後： 44


/Users/shinkaitomomichi/opt/anaconda3/envs/study-youtube/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/shinkaitomomichi/opt/anaconda3/envs/study-youtube/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
# 改めてアノテーションが正しいかを確認
filetered_videos = pd.read_csv('output/train/'+file_select+'/videos.csv', index_col=0)
HTML(output_html(filetered_videos, top_n=40))